# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI, a leading platform revolutionizing tech education. Offering expert-led courses in deep learning, neural networks, and more, DeepLearning.AI empowers learners to grasp complex AI concepts with ease. Whether you're a novice or seasoned professional, the platform caters to all skill levels. With practical insights and hands-on projects, mastering AI has never been more accessible. Join the millions upskilling with DeepLearning.AI and unlock boundless opportunities in the realm of artificial intelligence. Start your journey today and shape the future with DeepLearning.AI.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI, a leading platform revolutionizing tech education. Offering expert-led courses in deep learning, neural networks, and more, DeepLearning.AI empowers learners to grasp complex AI concepts with ease. Whether you're a novice or seasoned professional, the platform caters to all skill levels. With practical insights and hands-on projects, mastering AI has never been more accessible. Join the millions upskilling with DeepLearning.AI and unlock boundless opportunities in the realm of artificial intelligence. Start your journey today and shape the future with DeepLearning.AI.

------------------------------------------------

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI, a leading platform revolutionizing tech education. Offering expert-led courses in deep learning, neural networks, and more, DeepLearning.AI empowers learners to grasp complex AI concepts with ease. Whether you're a novice or seasoned professional, the platform caters to all skill levels. With practical insights and hands-on projects, mastering AI has never been more accessible. Join the millions upskilling with DeepLearning.AI and unlock boundless opportunities in the realm of artificial intelligence. Start your journey today and shape the future with DeepLearning.AI.

------------------------------------------------


--------------------------------------------------------------------------------
Critic (to Writer):

Aggregated Feedback: 
1. SEO Reviewer: Suggests improving SEO through keyword integration, meta tags optimization, and adding a clear call-to-action.
2. Legal Reviewer: Recommends refining the title to avoid trademark infringement, substantiating claims, and verifying content accuracy to avoid misleading information.
3. Ethics Reviewer: Emphasizes the importance of transparency regarding limitations and biases in AI education, backing up claims with evidence, and addressing conflicts of interest or undisclosed sponsorships that could impact objectivity.

Final Suggestions:
1. Enhance SEO by implementing suggested improvements like keyword integration, meta tags optimization, and incorporating a clear call-to-action.
2. Refine the title to ensure it does not infringe any trademarks, substantiate all claims, and verify the accuracy of content to avoid any misleading information.
3. Ensu

## Get the summary

In [15]:
print(res.summary)

Title: Navigating the World of DeepLearning.AI

Embark on a journey into the realm of artificial intelligence through DeepLearning.AI – a premier platform reshaping tech education landscapes. Featuring comprehensive courses on deep learning and neural networks, DeepLearning.AI equips learners of all levels with invaluable AI expertise. From beginners to professionals, the platform caters to diverse skill sets, offering practical knowledge and engaging projects. Join the vast community of learners mastering AI with DeepLearning.AI to unveil limitless opportunities in this cutting-edge field. Start shaping the future today with DeepLearning.AI.
